# Using the rs-server catalog

This tutorial is meant the basic usage of the rs-server catalog service.

## Introduction

The rs-server catalog service exposes a REST API 
on the /catalog endpoint of the rs-server.

Each user owns a dedicated STAC catalog.
It can be accessed at the /catalog/{user} endpoint.
Each catalog is following the STAC API.

Let's illustrate all this with some examples.

## Quick links

**Swagger UI**

  * http://localhost:8003/api.html (local)
  * https://dev-rspy.esa-copernicus.eu (cluster)

**STAC browser**

Open the STAC browser: http://localhost:8081 (only for local and hybrid mode)

Then load the collections (at the end of this demo but before deleting them) from: 

  * `http://localhost:8003/catalog/any` (local)
  * `https://dev-rspy.esa-copernicus.eu/catalog/any?api-key=your_api_key` (cluster)

## Configuration

In [1]:
# Install rs-client-libraries
!pip install rs_client_libraries-0.0.0-py3-none-any.whl

import getpass
import os

# S3 access
if not os.getenv("S3_ACCESSKEY"):
    os.environ["S3_ACCESSKEY"] = getpass.getpass(f"Enter S3 access key for {os.environ['S3_ENDPOINT']!r}:")
if not os.getenv("S3_SECRETKEY"):
    os.environ["S3_SECRETKEY"] = getpass.getpass(f"Enter S3 secret key for {os.environ['S3_ENDPOINT']!r}:")

# API key authentication (not on local mode)
if (os.getenv("RSPY_LOCAL_MODE") != "1") and (not os.getenv("RSPY_APIKEY")):
    os.environ["RSPY_APIKEY"] = getpass.getpass(f"Enter your API key from {os.environ['RSPY_WEBSITE']!r}:")

Defaulting to user installation because normal site-packages is not writeable
Processing ./rs_client_libraries-0.0.0-py3-none-any.whl
rs-client-libraries is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


Enter S3 access key for 'https://oss.eu-west-0.prod-cloud-ocb.orange-business.com': ········
Enter S3 secret key for 'https://oss.eu-west-0.prod-cloud-ocb.orange-business.com': ········
Enter your API key from 'https://dev-rspy.esa-copernicus.eu': ········


In [2]:
# Set local or cluster configuration
import os

if os.getenv("RSPY_LOCAL_MODE") == "1":
    RS_SERVER_ROOT_URL = "http://rs-server-catalog:8000"
    HEADERS={}
    local_mode = True
else:
    RS_SERVER_ROOT_URL = os.environ["RSPY_WEBSITE"]
    HEADERS={"headers": {"x-api-key": os.environ["RSPY_APIKEY"]}}
    local_mode = False

print(f"Using: {RS_SERVER_ROOT_URL}")

import requests
import json

Using: https://dev-rspy.esa-copernicus.eu


In [3]:
# Use boto3 for S3 operations

TEMP_BUCKET = "rs-cluster-temp"
FINAL_BUCKET = "rs-cluster-catalog"

!pip install boto3
import boto3
import os
s3_session = boto3.session.Session()

s3_client = s3_session.client(
    service_name="s3",    
    # Note: the S3_ACCESSKEY, S3_SECRETKEY and S3_ENDPOINT are given 
    # in the docker-compose.yml or ~/.s3cfg file.
    aws_access_key_id=os.environ["S3_ACCESSKEY"],
    aws_secret_access_key=os.environ["S3_SECRETKEY"],
    endpoint_url=os.environ["S3_ENDPOINT"],
    region_name=os.environ["S3_REGION"],
)
    
# Create the buckets if they don't exist (only in local mode)
if local_mode:
    buckets = [TEMP_BUCKET, FINAL_BUCKET] # bucket names under S3_ENDPOINT
    for b in buckets:
        if b not in [bucket["Name"] for bucket in s3_client.list_buckets()["Buckets"]]:
            s3_client.create_bucket(Bucket=b)

Defaulting to user installation because normal site-packages is not writeable


In [13]:
# Clean previous executions
# Delete collections
for collection in ("jgaucher:S1_L1", "jgaucher:S1_L2", "DemoUser:S1_L1", "ycolera:S1_L1"):
    requests.delete(f"{RS_SERVER_ROOT_URL}/catalog/collections/{collection}", **HEADERS, stream=True)

# Delete bucket files
for item_name in ("cadip/item1.dataset", "cadip/item2.dataset"):
    for bucket in (TEMP_BUCKET, FINAL_BUCKET):
        s3_client.delete_object(Bucket=bucket, Key=item_name)

In [14]:
# Create empty dummy data files for this demo
import tempfile
for item_name in ("cadip/item1.dataset", "cadip/item2.dataset"):
    with tempfile.NamedTemporaryFile() as tmp:
        s3_client.upload_file (tmp.name, TEMP_BUCKET, item_name)

# Save the current datetime (no ms, no time zone)
from datetime import datetime
demo_start_time = datetime.now().replace(microsecond=0).astimezone()

## Add the collection S1_L1 in the DemoUser catalog

In [15]:
collection = {
            "id": "S1_L1",
            "type": "Collection",
            "description": "The S1_L1 collection for DemoUser user.",
            "stac_version": "1.0.0",
            "owner": "DemoUser"
        }

post_response = requests.post(f"{RS_SERVER_ROOT_URL}/catalog/collections", json=collection, **HEADERS)
post_response.raise_for_status()

response = requests.get(f"{RS_SERVER_ROOT_URL}/catalog/collections/DemoUser:S1_L1", **HEADERS)
response.raise_for_status()

collection = json.loads(response.content)
collection

{'id': 'S1_L1',
 'type': 'Collection',
 'owner': 'DemoUser',
 'description': 'The S1_L1 collection for DemoUser user.',
 'stac_version': '1.0.0',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://dev-rspy.esa-copernicus.eu/catalog/DemoUser/collections/S1_L1/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://dev-rspy.esa-copernicus.eu/catalog/DemoUser'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://dev-rspy.esa-copernicus.eu/catalog/DemoUser'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://dev-rspy.esa-copernicus.eu/catalog/DemoUser/collections/S1_L1'}]}

The previous commands create the S1_L1 collection in the esmeralda catalog.
Then, it gets the esmeralda collections again.
This time, there is the S1_L1 collection, previously added.

As we can see, the user doesn't have to add the owner_id to the collection_id. It is automatically added during the post.

## Add the collection S1_L1 in the jgaucher catalog

In [16]:
collection = {
            "id": "S1_L1",
            "type": "Collection",
            "description": "The S1_L1 collection for jgaucher user.",
            "stac_version": "1.0.0",
            "owner": "jgaucher"
        }

post_response = requests.post(f"{RS_SERVER_ROOT_URL}/catalog/collections", json=collection, **HEADERS)
post_response.raise_for_status()

response = requests.get(f"{RS_SERVER_ROOT_URL}/catalog/collections/jgaucher:S1_L1", **HEADERS)
response.raise_for_status()

collection = json.loads(response.content)
collection

{'id': 'S1_L1',
 'type': 'Collection',
 'owner': 'jgaucher',
 'description': 'The S1_L1 collection for jgaucher user.',
 'stac_version': '1.0.0',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://dev-rspy.esa-copernicus.eu/catalog/jgaucher/collections/S1_L1/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://dev-rspy.esa-copernicus.eu/catalog/jgaucher'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://dev-rspy.esa-copernicus.eu/catalog/jgaucher'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://dev-rspy.esa-copernicus.eu/catalog/jgaucher/collections/S1_L1'}]}

Preview the content of catalog and temporary bucket, pushing a feature will automatically transfer files to catalog bucket, and update links.

## Add an item in the collection S1_L1 in the jgaucher catalog

In [17]:
item_0 = {
            "id": "item_0",
            "bbox": [-94.6334839, 37.0332547, -94.6005249, 37.0595608],
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [-94.6334839, 37.0595608],
                        [-94.6334839, 37.0332547],
                        [-94.6005249, 37.0332547],
                        [-94.6005249, 37.0595608],
                        [-94.6334839, 37.0595608],
                    ]
                ],
            },
            "collection": "S1_L1",
            "properties": {
                "gsd": 0.5971642834779395,
                "width": 2500,
                "height": 2500,
                "datetime": "2000-02-02T00:00:00Z",
                "proj:epsg": 3857,
                "orientation": "nadir",
                "owner_id": "jgaucher",
            },
            "stac_extensions": [],
            "assets": {
                "file": {
                    "href": "s3://rs-cluster-temp/cadip/item1.dataset",
                    "type": "image/tiff; application=geotiff; profile=cloud-optimized",
                    "title": "NOAA STORM COG",
                },
            },
        }

post_response = requests.post(f"{RS_SERVER_ROOT_URL}/catalog/collections/jgaucher:S1_L1/items", json=item_0, **HEADERS)
post_response.raise_for_status()

item_response = requests.get(f"{RS_SERVER_ROOT_URL}/catalog/collections/jgaucher:S1_L1/items", **HEADERS)
item_response.raise_for_status()

item = json.loads(item_response.content)
item

{'type': 'FeatureCollection',
 'context': {'limit': 10, 'returned': 1},
 'features': [{'id': 'item_0',
   'bbox': [-94.6334839, 37.0332547, -94.6005249, 37.0595608],
   'type': 'Feature',
   'assets': {'file': {'href': 'https://rs-server/catalog/jgaucher/collections/S1_L1/items/item1.dataset/download/file',
     'type': 'image/tiff; application=geotiff; profile=cloud-optimized',
     'title': 'NOAA STORM COG',
     'alternate': {'s3': {'href': 's3://rs-cluster-catalog/cadip/item1.dataset'}}}},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-94.6334839, 37.0595608],
      [-94.6334839, 37.0332547],
      [-94.6005249, 37.0332547],
      [-94.6005249, 37.0595608],
      [-94.6334839, 37.0595608]]]},
   'collection': 'S1_L1',
   'properties': {'gsd': 0.5971642834779395,
    'owner': 'jgaucher',
    'width': 2500,
    'height': 2500,
    'datetime': '2000-02-02T00:00:00Z',
    'owner_id': 'jgaucher',
    'proj:epsg': 3857,
    'orientation': 'nadir'},
   'stac_extensions': ['http

Preview again the content of buckets, to make sure that assests were moved.

In [18]:
# Check that the asset was copied to the final catalog bucket
item1_time = s3_client.head_object(Bucket=FINAL_BUCKET, Key="cadip/item1.dataset")["LastModified"]
assert item1_time >= demo_start_time, \
       f"'{item1_time}' should be >= '{demo_start_time}'"

# And was removed from the temp bucket
assert "Contents" not in s3_client.list_objects(Bucket=TEMP_BUCKET, Prefix="cadip/item1.dataset"), \
       "Should have been removed from the temp bucket"

The previous commands create the first feature in the esmeralda S1_L1 collection. Then, it gets the the feature created.

Let's look at its content.

## Add an item in the collection S1_L1 in the DemoUser catalog

In [19]:
item_0 = {
            "id": "item_0",
            "bbox": [-94.6334839, 37.0332547, -94.6005249, 37.0595608],
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [-94.6334839, 37.0595608],
                        [-94.6334839, 37.0332547],
                        [-94.6005249, 37.0332547],
                        [-94.6005249, 37.0595608],
                        [-94.6334839, 37.0595608],
                    ]
                ],
            },
            "collection": "S1_L1",
            "properties": {
                "gsd": 0.5971642834779395,
                "width": 2500,
                "height": 2500,
                "datetime": "2000-02-02T00:00:00Z",
                "proj:epsg": 3857,
                "orientation": "nadir",
                "owner_id": "DemoUser",
            },
            "stac_extensions": [],
            "assets": {
                "file": {
                    "href": "s3://rs-cluster-temp/cadip/item2.dataset",
                    "type": "image/tiff; application=geotiff; profile=cloud-optimized",
                    "title": "NOAA STORM COG",
                },
            },
        }

post_response = requests.post(f"{RS_SERVER_ROOT_URL}/catalog/collections/DemoUser:S1_L1/items", json=item_0, **HEADERS)
post_response.raise_for_status()

item_response = requests.get(f"{RS_SERVER_ROOT_URL}/catalog/collections/DemoUser:S1_L1/items", **HEADERS)
item_response.raise_for_status()

item = json.loads(item_response.content)
item

{'type': 'FeatureCollection',
 'context': {'limit': 10, 'returned': 1},
 'features': [{'id': 'item_0',
   'bbox': [-94.6334839, 37.0332547, -94.6005249, 37.0595608],
   'type': 'Feature',
   'assets': {'file': {'href': 'https://rs-server/catalog/DemoUser/collections/S1_L1/items/item2.dataset/download/file',
     'type': 'image/tiff; application=geotiff; profile=cloud-optimized',
     'title': 'NOAA STORM COG',
     'alternate': {'s3': {'href': 's3://rs-cluster-catalog/cadip/item2.dataset'}}}},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-94.6334839, 37.0595608],
      [-94.6334839, 37.0332547],
      [-94.6005249, 37.0332547],
      [-94.6005249, 37.0595608],
      [-94.6334839, 37.0595608]]]},
   'collection': 'S1_L1',
   'properties': {'gsd': 0.5971642834779395,
    'owner': 'DemoUser',
    'width': 2500,
    'height': 2500,
    'datetime': '2000-02-02T00:00:00Z',
    'owner_id': 'DemoUser',
    'proj:epsg': 3857,
    'orientation': 'nadir'},
   'stac_extensions': ['http

## Get the first item from jgaucher S1_L1 collection

In [20]:
import requests
import json

get_response = requests.get(f"{RS_SERVER_ROOT_URL}/catalog/collections/jgaucher:S1_L1/items/item_0", **HEADERS)
get_response.raise_for_status()

item = json.loads(get_response.content)
item

{'id': 'item_0',
 'bbox': [-94.6334839, 37.0332547, -94.6005249, 37.0595608],
 'type': 'Feature',
 'assets': {'file': {'href': 'https://rs-server/catalog/jgaucher/collections/S1_L1/items/item1.dataset/download/file',
   'type': 'image/tiff; application=geotiff; profile=cloud-optimized',
   'title': 'NOAA STORM COG',
   'alternate': {'s3': {'href': 's3://rs-cluster-catalog/cadip/item1.dataset'}}}},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-94.6334839, 37.0595608],
    [-94.6334839, 37.0332547],
    [-94.6005249, 37.0332547],
    [-94.6005249, 37.0595608],
    [-94.6334839, 37.0595608]]]},
 'collection': 'S1_L1',
 'properties': {'gsd': 0.5971642834779395,
  'owner': 'jgaucher',
  'width': 2500,
  'height': 2500,
  'datetime': '2000-02-02T00:00:00Z',
  'owner_id': 'jgaucher',
  'proj:epsg': 3857,
  'orientation': 'nadir'},
 'stac_extensions': ['https://stac-extensions.github.io/alternate-assets/v1.1.0/schema.json'],
 'links': [{'rel': 'collection',
   'type': 'application/json

The previous commands will display a single item from esmeralda S1_L1 collection

## Add 2 new items in jgaucher S1_L1 collection and search all items with the datetime: 

In [21]:
item_1 = {
            "id": "item_1",
            "bbox": [-94.6334839, 37.0332547, -94.6005249, 37.0595608],
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [-94.6334839, 37.0595608],
                        [-94.6334839, 37.0332547],
                        [-94.6005249, 37.0332547],
                        [-94.6005249, 37.0595608],
                        [-94.6334839, 37.0595608],
                    ]
                ],
            },
            "collection": "S1_L1",
            "properties": {
                "gsd": 0.5971642834779395,
                "width": 2500,
                "height": 2500,
                "datetime": "2000-03-02T00:00:00Z",
                "proj:epsg": 3857,
                "orientation": "nadir",
                "owner_id": "jgaucher",
            },
            "stac_extensions": [],
            "assets": {
                "file": {
                    "href": "s3://rs-cluster-temp/cadip/item1.dataset",
                    "type": "image/tiff; application=geotiff; profile=cloud-optimized",
                    "title": "NOAA STORM COG",
                },
            },
        }

item_2 = {
            "id": "item_2",
            "bbox": [-94.6334839, 37.0332547, -94.6005249, 37.0595608],
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [-94.6334839, 37.0595608],
                        [-94.6334839, 37.0332547],
                        [-94.6005249, 37.0332547],
                        [-94.6005249, 37.0595608],
                        [-94.6334839, 37.0595608],
                    ]
                ],
            },
            "collection": "S1_L1",
            "properties": {
                "gsd": 0.5971642834779395,
                "width": 2500,
                "height": 2500,
                "datetime": "2000-03-02T00:00:00Z",
                "proj:epsg": 3857,
                "orientation": "nadir",
                "owner_id": "jgaucher",
            },
            "stac_extensions": [],
            "assets": {
                "file": {
                    "href": "s3://rs-cluster-temp/cadip/item2.dataset",
                    "type": "image/tiff; application=geotiff; profile=cloud-optimized",
                    "title": "NOAA STORM COG",
                },
            },
        }
# Create empty dummy data files for this demo
import tempfile
for item_name in ("cadip/item1.dataset", "cadip/item2.dataset"):
    with tempfile.NamedTemporaryFile() as tmp:
        s3_client.upload_file (tmp.name, TEMP_BUCKET, item_name)

# Save the current datetime (no ms, no time zone)
from datetime import datetime
demo_start_time = datetime.now().replace(microsecond=0).astimezone()
post = requests.post(f"{RS_SERVER_ROOT_URL}/catalog/collections/jgaucher:S1_L1/items", json=item_1, **HEADERS)
post.raise_for_status()
post = requests.post(f"{RS_SERVER_ROOT_URL}/catalog/collections/jgaucher:S1_L1/items", json=item_2, **HEADERS)
post.raise_for_status()

In [22]:
parameters = {"collections": ["S1_L1"], "filter": "datetime='2000-03-02T00:00:00Z' AND owner_id='jgaucher'"}
search_response = requests.get(f"{RS_SERVER_ROOT_URL}/catalog/search", params=parameters, **HEADERS)
search_response.raise_for_status()

result = json.loads(search_response.content)
result

{'type': 'FeatureCollection',
 'context': {'limit': 10, 'returned': 2},
 'features': [{'id': 'item_2',
   'bbox': [-94.6334839, 37.0332547, -94.6005249, 37.0595608],
   'type': 'Feature',
   'assets': {'file': {'href': 'https://rs-server/catalog/jgaucher/collections/S1_L1/items/item2.dataset/download/file',
     'type': 'image/tiff; application=geotiff; profile=cloud-optimized',
     'title': 'NOAA STORM COG',
     'alternate': {'s3': {'href': 's3://rs-cluster-catalog/cadip/item2.dataset'}}}},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-94.6334839, 37.0595608],
      [-94.6334839, 37.0332547],
      [-94.6005249, 37.0332547],
      [-94.6005249, 37.0595608],
      [-94.6334839, 37.0595608]]]},
   'collection': 'S1_L1',
   'properties': {'gsd': 0.5971642834779395,
    'owner': 'jgaucher',
    'width': 2500,
    'height': 2500,
    'datetime': '2000-03-02T00:00:00Z',
    'owner_id': 'jgaucher',
    'proj:epsg': 3857,
    'orientation': 'nadir'},
   'stac_extensions': ['http

The previous command will display the result from the search endpoint with specifics parameters such as the collection name or the datetime.

In [23]:
json_parameters = {
    "collections": ["S1_L1"],
    "filter-lang": "cql2-json",
    "filter": {
        "op": "and",
        "args": [
            {"op": "=", "args": [{"property": "owner_id"}, "jgaucher"]},
            {"op": "=", "args": [{"property": "datetime"}, "2000-03-02T00:00:00Z"]},
        ],
    },
}

search_response = requests.post(f"{RS_SERVER_ROOT_URL}/catalog/search", json=json_parameters, **HEADERS)
search_response.raise_for_status()

result = json.loads(search_response.content)
result

{'type': 'FeatureCollection',
 'context': {'limit': 10, 'returned': 2},
 'features': [{'id': 'item_2',
   'bbox': [-94.6334839, 37.0332547, -94.6005249, 37.0595608],
   'type': 'Feature',
   'assets': {'file': {'href': 'https://rs-server/catalog/jgaucher/collections/S1_L1/items/item2.dataset/download/file',
     'type': 'image/tiff; application=geotiff; profile=cloud-optimized',
     'title': 'NOAA STORM COG',
     'alternate': {'s3': {'href': 's3://rs-cluster-catalog/cadip/item2.dataset'}}}},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-94.6334839, 37.0595608],
      [-94.6334839, 37.0332547],
      [-94.6005249, 37.0332547],
      [-94.6005249, 37.0595608],
      [-94.6334839, 37.0595608]]]},
   'collection': 'S1_L1',
   'properties': {'gsd': 0.5971642834779395,
    'owner': 'jgaucher',
    'width': 2500,
    'height': 2500,
    'datetime': '2000-03-02T00:00:00Z',
    'owner_id': 'jgaucher',
    'proj:epsg': 3857,
    'orientation': 'nadir'},
   'stac_extensions': ['http

We can do the same operation using a post request.

## Update the item_0 from the jgaucher S1_L1 collection

In [24]:
new_item_0 = {
            "id": "item_0",
            "bbox": [-94.6334839, 37.0332547, -94.6005249, 37.0595608],
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [-100, 37.0595608],
                        [-108, 37.0332547],
                        [-100, 37.0332547],
                        [-111, 37.0595608],
                        [-100, 37.0595608],
                    ]
                ],
            },
            "collection": "S1_L1",
            "properties": {
                "gsd": 0.5971642834779395,
                "width": 5000,
                "height": 2500,
                "datetime": "2014-02-02T00:00:00Z",
                "proj:epsg": 3857,
                "orientation": "nadir",
                "owner_id": "jgaucher",
            },
            "stac_extensions": [],
            "assets": {
                "file": {
                    "href": "s3://rs-cluster-temp/cadip/item2.dataset",
                    "type": "image/tiff; application=geotiff; profile=cloud-optimized",
                    "title": "NOAA STORM COG",
                },
            },
        }
# Create empty dummy data files for this demo
import tempfile
for item_name in ("cadip/item1.dataset", "cadip/item2.dataset"):
    with tempfile.NamedTemporaryFile() as tmp:
        s3_client.upload_file (tmp.name, TEMP_BUCKET, item_name)
        
update_response = requests.put(f"{RS_SERVER_ROOT_URL}/catalog/collections/jgaucher:S1_L1/items/item_0", json=new_item_0, **HEADERS)
update_response.raise_for_status()

result = json.loads(update_response.content)
result

{'type': 'Feature',
 'stac_extensions': ['https://stac-extensions.github.io/alternate-assets/v1.1.0/schema.json'],
 'id': 'item_0',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-100, 37.0595608],
    [-108, 37.0332547],
    [-100, 37.0332547],
    [-111, 37.0595608],
    [-100, 37.0595608]]]},
 'bbox': [-94.6334839, 37.0332547, -94.6005249, 37.0595608],
 'properties': {'gsd': 0.5971642834779395,
  'width': 5000,
  'height': 2500,
  'datetime': '2014-02-02T00:00:00Z',
  'proj:epsg': 3857,
  'orientation': 'nadir',
  'owner_id': 'jgaucher',
  'owner': 'jgaucher'},
 'assets': {'file': {'href': 'https://rs-server/catalog/jgaucher/collections/S1_L1/items/item2.dataset/download/file',
   'type': 'image/tiff; application=geotiff; profile=cloud-optimized',
   'title': 'NOAA STORM COG',
   'alternate': {'s3': {'href': 's3://rs-cluster-catalog/cadip/item2.dataset'}}}},
 'collection': 'jgaucher_S1_L1',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://dev

The previous command will update the item_0 from the Esmeralda S1_L1 collection by changing geometry, width and datetime.

In [25]:
# Check that the asset was copied to the final catalog bucket
item1_time = s3_client.head_object(Bucket=FINAL_BUCKET, Key="cadip/item2.dataset")["LastModified"]
assert item1_time >= demo_start_time, \
       f"'{item1_time}' should be >= '{demo_start_time}'"

# And was removed from the temp bucket
assert "Contents" not in s3_client.list_objects(Bucket=TEMP_BUCKET, Prefix="cadip/item2.dataset"), \
       "Should have been removed from the temp bucket"

## Update the collection S1_L1 from the jgaucher catalog

In [26]:
new_collection_S1_L1 = {
            "id": "S1_L1",
            "type": "Collection",
            "description": "This is the new description for the S1_L1 collection for jgaucher user.",
            "stac_version": "1.0.0",
            "owner": "jgaucher"
        }

update_response = requests.put(f"{RS_SERVER_ROOT_URL}/catalog/collections", json=new_collection_S1_L1, **HEADERS)
update_response.raise_for_status()

result = json.loads(update_response.content)
result

{'id': 'S1_L1',
 'type': 'Collection',
 'description': 'This is the new description for the S1_L1 collection for jgaucher user.',
 'stac_version': '1.0.0',
 'owner': 'jgaucher',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://dev-rspy.esa-copernicus.eu/catalog/jgaucher/collections/S1_L1/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://dev-rspy.esa-copernicus.eu/catalog/jgaucher'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://dev-rspy.esa-copernicus.eu/catalog/jgaucher'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://dev-rspy.esa-copernicus.eu/catalog/jgaucher/collections/S1_L1'}]}

The previous command will update the collection S1_L1 from the Esmeralda catalog by changing the collection description.

## Get all the accessible collections from the user ycolera

In [27]:
response = requests.get(f"{RS_SERVER_ROOT_URL}/catalog/collections", **HEADERS)
response.raise_for_status()

json.loads(response.content)

{'collections': [{'id': 'toto_sentinel1-grd',
   'type': 'Collection',
   'links': [{'rel': 'items',
     'type': 'application/geo+json',
     'href': 'https://dev-rspy.esa-copernicus.eu/collections/toto_sentinel1-grd/items'},
    {'rel': 'parent',
     'type': 'application/json',
     'href': 'https://dev-rspy.esa-copernicus.eu/'},
    {'rel': 'root',
     'type': 'application/json',
     'href': 'https://dev-rspy.esa-copernicus.eu/'},
    {'rel': 'self',
     'type': 'application/json',
     'href': 'https://dev-rspy.esa-copernicus.eu/collections/toto_sentinel1-grd'},
    {'rel': 'license',
     'href': 'https://scihub.copernicus.eu/twiki/do/view/SciHubWebPortal/TermsConditions',
     'title': 'Sentinel License'},
    {'rel': 'about',
     'href': 'https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-1-sar/products-algorithms/level-1-algorithms/ground-range-detected',
     'title': 'Sentinel-1 Ground Range Detected (GRD) Technical Guide'}],
   'title': 'Sentinel-1 G

The previous command show all the collection that the user "ycolera" has access to.

## Get the landing page

In [28]:
response = requests.get(f"{RS_SERVER_ROOT_URL}/catalog/", **HEADERS)
response.raise_for_status()

landing_page = json.loads(response.content)
landing_page

{'type': 'Catalog',
 'id': 'stac-fastapi',
 'title': 'stac-fastapi',
 'description': 'stac-fastapi',
 'stac_version': '1.0.0',
 'conformsTo': ['https://api.stacspec.org/v1.0.0/ogcapi-features',
  'https://api.stacspec.org/v1.0.0/item-search#fields',
  'https://api.stacspec.org/v1.0.0/item-search',
  'http://www.opengis.net/spec/cql2/1.0/conf/cql2-json',
  'http://www.opengis.net/spec/cql2/1.0/conf/cql2-text',
  'https://api.stacspec.org/v1.0.0/item-search#query',
  'https://api.stacspec.org/v1.0.0/item-search#sort',
  'https://api.stacspec.org/v1.0.0-rc.2/item-search#context',
  'https://api.stacspec.org/v1.0.0/collections',
  'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/core',
  'https://api.stacspec.org/v1.0.0-rc.2/item-search#filter',
  'http://www.opengis.net/spec/cql2/1.0/conf/basic-cql2',
  'http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/features-filter',
  'https://api.stacspec.org/v1.0.0/core',
  'https://api.stacspec.org/v1.0.0-rc.3/ogcapi-features/extension

The previous command loads the landing page of the user "ycolera". We can see all the collections and catalogs he has access to.

## Using an endpoint without being authenticated

In [29]:
response = requests.get(f"{RS_SERVER_ROOT_URL}/catalog/")

landing_page = json.loads(response.content)
landing_page

{'code': 'HTTPException', 'description': '403: Not authenticated'}

## Using an endpoint without being authorized

In [30]:
toto_S1_L1 = {
            "id": "S1_L1",
            "type": "Collection",
            "description": "This is the new description for the S1_L1 collection for toto user.",
            "stac_version": "1.0.0",
            "owner": "toto"
        }

response = requests.post(f"{RS_SERVER_ROOT_URL}/catalog/collections", json=toto_S1_L1, **HEADERS)

content = json.loads(response.content)
content

{'error': 'Unauthorized access.'}

## Get the jgaucher catalog

In [31]:
response = requests.get(f"{RS_SERVER_ROOT_URL}/catalog/catalogs/jgaucher", **HEADERS)
response.raise_for_status()

catalog_jgaucher = json.loads(response.content)
catalog_jgaucher

{'type': 'Catalog',
 'id': 'stac-fastapi',
 'title': 'stac-fastapi',
 'description': 'stac-fastapi',
 'stac_version': '1.0.0',
 'conformsTo': ['https://api.stacspec.org/v1.0.0/ogcapi-features',
  'https://api.stacspec.org/v1.0.0/item-search#fields',
  'https://api.stacspec.org/v1.0.0/item-search',
  'http://www.opengis.net/spec/cql2/1.0/conf/cql2-json',
  'http://www.opengis.net/spec/cql2/1.0/conf/cql2-text',
  'https://api.stacspec.org/v1.0.0/item-search#query',
  'https://api.stacspec.org/v1.0.0/item-search#sort',
  'https://api.stacspec.org/v1.0.0-rc.2/item-search#context',
  'https://api.stacspec.org/v1.0.0/collections',
  'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/core',
  'https://api.stacspec.org/v1.0.0-rc.2/item-search#filter',
  'http://www.opengis.net/spec/cql2/1.0/conf/basic-cql2',
  'http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/features-filter',
  'https://api.stacspec.org/v1.0.0/core',
  'https://api.stacspec.org/v1.0.0-rc.3/ogcapi-features/extension

## Get an unauthorized catalog

In [32]:
response = requests.get(f"{RS_SERVER_ROOT_URL}/catalog/catalogs/titi", **HEADERS)

catalog_jgaucher = json.loads(response.content)
catalog_jgaucher

{'error': 'Unauthorized access.'}

## The user_login linked to the API key has implicit roles on his own catalog

In [33]:
ycolera_collection = {
            "id": "S1_L1",
            "type": "Collection",
            "description": "This is the  description for the S1_L1 collection for ycolera user.",
            "stac_version": "1.0.0",
            "owner": "ycolera"
        }
post_new_collection = requests.post(f"{RS_SERVER_ROOT_URL}/catalog/collections", json=ycolera_collection, **HEADERS)
post_new_collection.raise_for_status()

In [34]:
response = requests.get(f"{RS_SERVER_ROOT_URL}/catalog/collections/ycolera:S1_L1", **HEADERS)
response.raise_for_status()

content = json.loads(response.content)
content

{'id': 'S1_L1',
 'type': 'Collection',
 'owner': 'ycolera',
 'description': 'This is the  description for the S1_L1 collection for ycolera user.',
 'stac_version': '1.0.0',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://dev-rspy.esa-copernicus.eu/catalog/ycolera/collections/S1_L1/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://dev-rspy.esa-copernicus.eu/catalog/ycolera'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://dev-rspy.esa-copernicus.eu/catalog/ycolera'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://dev-rspy.esa-copernicus.eu/catalog/ycolera/collections/S1_L1'}]}

## Delete an item from the jgaucher S1_L1 collection

In [35]:
delete_response = requests.delete(f"{RS_SERVER_ROOT_URL}/catalog/collections/jgaucher:S1_L1/items/item_2", **HEADERS)
delete_response.raise_for_status()

result = json.loads(delete_response.content)
result

{'deleted item': 'item_2'}

## Delete the collection S1_L1 from the jgaucher catalog

In [36]:
delete_response = requests.delete(f"{RS_SERVER_ROOT_URL}/catalog/collections/jgaucher:S1_L1", **HEADERS)
delete_response.raise_for_status()

result = json.loads(delete_response.content)
result

{'deleted collection': 'jgaucher_S1_L1'}

## Delete the collection S1_L1 from the DemoUser catalog

In [39]:
delete_response = requests.delete(f"{RS_SERVER_ROOT_URL}/catalog/collections/DemoUser:S1_L1", **HEADERS)
delete_response.raise_for_status()

result = json.loads(delete_response.content)
result

ChunkedEncodingError: ('Connection broken: IncompleteRead(0 bytes read, 41 more expected)', IncompleteRead(0 bytes read, 41 more expected))

## Clear files from catalog bucket

In [38]:
for item_name in ("cadip/item1.dataset", "cadip/item2.dataset"):
    for bucket in (TEMP_BUCKET, FINAL_BUCKET):
        s3_client.delete_object(Bucket=bucket, Key=item_name)